<a href="https://colab.research.google.com/github/karasu1982/TwitterAPI/blob/main/TwitterAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境設定

In [ ]:
!pip install tweepy==4.0.0

In [ ]:
!pip install emoji

     |████████████████████████████████| 174 kB 14.7 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=8815ed5850c529cf17e9e4c98b71e8e7b3d75fc20c99a2d513bda1345100a28e
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [ ]:
# ライブラリのインポート
import tweepy
from datetime import datetime,timezone
import pytz
import pandas as pd

import requests
import re
import emoji

# データ取得・加工


## Twitterデータ取得

In [ ]:
# Twitter TOKEN
BEARER_TOKEN = "<Twitter APIキー>"

# Twitter APIのURL
search_url = "https://api.twitter.com/2/tweets/search/recent"

In [ ]:
# 検索クエリ
query_params = {'query': '保育士 -is:retweet', 'tweet.fields': 'author_id', 'max_results': 100}

In [ ]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers, params):
    has_next = True
    c = 0
    result = []
    while has_next:
        response = requests.request("GET", search_url, headers=headers, params=params)
        if response.status_code != 200:
            raise Exception(response.status_code, response.text)

        response_body = response.json()
        result += response_body['data']

        rate_limit = response.headers['x-rate-limit-remaining']
        print('Rate limit remaining: ' + rate_limit)

        c = c + 1
        has_next = ('next_token' in response_body['meta'].keys() and c < 300)

        # next_tokenがある場合は検索クエリに追加
        if has_next:
            query_params['next_token'] = response_body['meta']['next_token']

    return result

# Pandasのデータフレームを使ってデータを加工
def create_data_frame(json_data):


In [ ]:
headers = create_headers(BEARER_TOKEN)
json_response = connect_to_endpoint(search_url, headers, query_params)

## データ加工

In [ ]:
import re

def preprocess_word(doc):
  # 数字を０に 
  # doc = re.sub(r'\d+', '0', doc) 

  # メンション除去 
  doc = re.sub(r"@(\w+) ", "", doc) 

  # url除去 
  doc = re.sub(r"http\S+", "", doc)

  #リツイートを消す
  doc = re.sub(r"(^RT.*)", "", doc, flags=re.MULTILINE | re.DOTALL)

  # 大文字・小文字変換 
  doc = doc.lower() 

  #絵文字を消す
  emoji_pattern = re.compile("["
  u"\U0001F600-\U0001F64F"
  u"\U0001F300-\U0001F5FF"
  u"\U0001F680-\U0001F6FF"
  u"\U0001F1E0-\U0001F1FF"
  "]+", flags=re.UNICODE)
  doc = emoji_pattern.sub("", doc)

  # 不要記号削除
  pattern = '[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”◇ᴗ●↓→♪★⊂⊃※△□◎〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％]' 
  doc = re.sub(pattern, ' ', doc) 

  # 多すぎるワードは除く
  doc = doc.replace("保育士","")

  return doc

In [ ]:
df = pd.DataFrame(json_response)

df["text"] = df["text"].apply(preprocess_word)

## Googleドライブに保存

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
tmp_path = "/content/gdrive/MyDrive/Colab Notebooks/tmp/"

In [ ]:
df.to_csv(tmp_path+"twitter_text.csv")

In [ ]:
import pandas as pd
df = pd.read_csv(tmp_path+"twitter_text.csv")

In [ ]:
df["text"] = df["text"].apply(preprocess_word)

In [ ]:
df.head()

,Unnamed: 0,author_id,id,text,emotion
0,0,1090606035658625024,1490209147811659777,通りすがりのです色々な先輩と働いたのち 最近は子どもが保育園に来ているだけで褒めてあげたいと...,mostly_POSITIVE
1,1,1438337387550035969,1490209022674616322,世のさん腱鞘炎と戦いつつもがんばって仕事してるのえらいな…勤めてる最中になるぶんには大丈夫そ...,NEGATIVE
2,2,1373419650650759173,1490209005691899905,前向きに二人での海外旅行を企画したくても出来ない こんな窮屈な世の中早く終わってくれ 欲を...,NEUTRAL
3,3,100352197,1490208785796775942,だけど義務化では無いので肯定的です 推奨する分にはいいかな と現場の匙加減でどうにでも出来ますし,None
4,4,1260540744390938624,1490208653642964992,同じ嬉しい \nよろしくお願いします🤲,POSITIVE


# データ分析

## 環境設定

Mecabおよび日本語フォントのインストール

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.9)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.16)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+ma

In [ ]:
!apt -y install fonts-ipafont-gothic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-ipafont-gothic is already the newest version (00303-18ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
!pip install pymlask

## 感情分析

ML-Askを利用<br>
https://qiita.com/konitech913/items/317b452cc6c63894fce3

### ML-Ask実行

In [ ]:
from mlask import MLAsk
emotion_analyzer = MLAsk()
emotion_analyzer.analyze('彼のことは嫌い')

{'activation': 'NEUTRAL',
 'emoticon': None,
 'emotion': defaultdict(list, {'iya': ['嫌う']}),
 'intensifier': {},
 'intension': 0,
 'orientation': 'NEGATIVE',
 'representative': ('iya', ['嫌う']),
 'text': '彼のことは嫌い'}

In [ ]:
orientation_list = []

for i, d in df.iterrows():
  e = emotion_analyzer.analyze(d.text)

  if e["emotion"] is None:
    _e = None
  else:
    _e = e["orientation"]
  orientation_list.append(_e)

df["emotion"] = orientation_list

In [ ]:
df["emotion"].value_counts()

NEGATIVE           3335
POSITIVE           3139
NEUTRAL             900
mostly_NEGATIVE     497
mostly_POSITIVE     328
Name: emotion, dtype: int64

In [ ]:
df_positive = df[df["emotion"] == "POSITIVE"]
df_negative = df[df["emotion"] == "NEGATIVE"]

In [ ]:
df[df["emotion"] == "POSITIVE"]

,Unnamed: 0,author_id,id,text,emotion
4,4,1260540744390938624,1490208653642964992,同じ嬉しい \nよろしくお願いします🤲,POSITIVE
5,5,1253523190220943361,1490208573938614273,スッキリした人間関係\n高水準の給与\nほどほどの幸せを求めて 日々チームワークで助け合い仕...,POSITIVE
6,6,1364423333618458624,1490208132957622274,長袖 エプロン スモック 大人 割烹着 おしゃれ あったか 冬 起毛 t rチェック 袖リ...,POSITIVE
12,12,2938079040,1490207775619948555,さん尊敬でしかない🥺こんなやんちゃな子供を何十人も一気に面倒見て…🥺\n\n1週間の休園で疲...,POSITIVE
15,15,331824933,1490207388275986432,new未経験 目指す細身の可愛い女子大生 ／堺筋本町 プライベートサロン メンエス 羽衣天...,POSITIVE
...,...,...,...,...,...
29792,29792,1485147979208364035,1487999405026263040,セール エプロン おしゃれ tika ティカ ポケット付き バイカラー ハート フレアエプロ...,POSITIVE
29794,29794,2743272998,1487999227607207937,私の妹志望なんだけど 私の母が妹の小さい時の靴を見つけて履かせたらぴったりだったんだけど 色...,POSITIVE
29801,29801,1325082068380868611,1487998733770182656,試験で予防接種の問題出てくるからこのスケジュール表見てたけど 細かすぎてほぼ捨ててたなー笑笑...,POSITIVE
29802,29802,1456958171902525440,1487998615193010176,鬼めっちゃかわいい\nまきちゃんさん,POSITIVE


In [ ]:
# 見える化したいデータ
_df = df_negative.copy()

## 取得したデータを見える化(WordCloud)

https://qiita.com/turmericN/items/04cd0b40f91076f0ef42

In [ ]:
!pip install janome

In [ ]:
from janome.tokenizer import Tokenizer
from collections import Counter, defaultdict
from wordcloud import WordCloud

In [ ]:
font_path_gothic = '/usr/share/fonts/opentype/ipafont-gothic/ipagp.ttf'

In [ ]:
def counter(texts):
    t = Tokenizer()
    words_count = defaultdict(int)
    words = []
    for text in texts:
        tokens = t.tokenize(text)
        for token in tokens:
            # 品詞から名詞だけ抽出
            pos = token.part_of_speech.split(',')[0]
            if pos in ['名詞']:
                # 必要ない単語を省く(実際の結果を見てから不必要そうな単語を記載しました)
                if token.base_form not in ["こと", "よう", "そう", "これ", "それ"]:
                    words_count[token.base_form] += 1
                    words.append(token.base_form)
    return words_count, words

In [ ]:
words_count, words = counter(_df["text"])
text = ' '.join(words)

In [ ]:
wordcloud = WordCloud(background_color="white",
                      font_path=font_path_gothic, width=900, height=500).generate(text)

In [ ]:
wordcloud.to_file("./wordcloud_sample.png")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,10))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

## 取得したデータを見える化(ネットワーク図)

https://qiita.com/y_itoh/items/7aa33ba0b1e30b3ea33d

### Jaccard係数計算

In [ ]:
import MeCab
mecab = MeCab.Tagger("-Ochasen")

# 文単位の名詞リストを生成
noun_list = [
             [v.split()[2] for v in mecab.parse(sentence).splitlines()
             if (len(v.split())>=3 and v.split()[3][:2]=='名詞')]
             for sentence in _df["text"]
             ]

In [ ]:
from itertools import combinations, dropwhile
from collections import Counter, OrderedDict

def bform2pair(bform_2l, min_cnt=5):
    # 単語ペアの出現章数をカウント
 
    # 全単語ペアのリスト
    pair_all = []
 
    for bform_l in bform_2l:
        # 章ごとに単語ペアを作成
        # combinationsを使うと順番が違うだけのペアは重複しない
        # ただし、同単語のペアは存在しえるのでsetでユニークにする
        pair_l = list(combinations(set(bform_l), 2))
 
        # 単語ペアの順番をソート
        for i,pair in enumerate(pair_l):
            pair_l[i] = tuple(sorted(pair))
 
        pair_all += pair_l
 
    # 単語ペアごとの出現章数
    pair_count = Counter(pair_all)
 
    # ペア数がmin_cnt以上に限定
    for key, count in dropwhile(lambda key_count: key_count[1] >= min_cnt, pair_count.most_common()):
        del pair_count[key]
 
    return pair_count

In [ ]:
def pair2jaccard(pair_count, bform_2l, edge_th=0.4):
    # jaccard係数を計算
 
    # 単語ごとの出現章数
    word_count = Counter()
    for bform_l in bform_2l:
        word_count += Counter(set(bform_l))
 
    # 単語ペアごとのjaccard係数を計算
    jaccard_coef = []
    for pair, cnt in pair_count.items():
        jaccard_coef.append(cnt / (word_count[pair[0]] + word_count[pair[1]] - cnt))
 
    # jaccard係数がedge_th未満の単語ペアを除外
    jaccard_dict = OrderedDict()
    for (pair, cnt), coef in zip(pair_count.items(), jaccard_coef):
        if coef >= edge_th:
            jaccard_dict[pair] = coef
            #print(pair, cnt, coef, word_count[pair[0]], word_count[pair[1]], sep='\t')
 
    return jaccard_dict

In [ ]:
pair_count = bform2pair(noun_list, min_cnt=5)
jaccard_dict = pair2jaccard(pair_count, noun_list, edge_th=0.5) 

### ネットワーク図

https://irukanobox.blogspot.com/2021/02/python.html

#### 必要なソフトウェア・ライブラリ

In [ ]:
!sudo apt install graphviz
!sudo apt-get install -y python3-dev graphviz libgraphviz-dev pkg-config
!pip install pygraphviz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.40.1-2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
graphviz is already the newest version (2.40.1-2).
libgraphviz-dev is already the newest version (2.40.1-2).
python3-dev is already the newest version (3.6.7-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


#### ネットワーク図作成

In [ ]:
import networkx as nx
# matplotlibのターミナル対応
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import pygraphviz
import plotly.graph_objects as go

In [ ]:
def build_interactive_network(G, pos, node_sizes, node_colors):

    # edgeデータの作成
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
  
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')
  
    # nodeデータの作成
    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
  
    # nodeの色、サイズ、マウスオーバーしたときに表示するテキストの設定
    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        text=list(G.nodes()),
        hovertext=node_sizes,
        textposition='top center',
        mode='markers+text',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='Portland',
            reversescale=False,
            color=node_colors,
            size=node_sizes,
            colorbar=dict(
                thickness=15,
                title='Tweet数',
            ),
            line_width=2))
     
    data = [edge_trace, node_trace]
 
    # レイアウトの設定
    layout=go.Layout(
                paper_bgcolor='rgb(233,233,233)',
                plot_bgcolor='rgba(0,0,0,0)',
                showlegend=False,
                hovermode='closest',
                margin=dict(b=10, l=5, r=5, t=10),
                font=dict(size=10),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis = dict(showgrid = False, zeroline = False, showticklabels = False))
 
    fig = go.Figure(data=data, layout=layout)
    return fig

In [ ]:
import itertools
import collections

_n = list(itertools.chain.from_iterable(noun_list))
#nodes = set(_n)
_c = collections.Counter(_n)

# 件数が多いため100で割る
dict_c = dict(zip(list(_c.keys()), [n/10 for n in _c.values()]))

In [ ]:
freq_top=80
edge_top=60
 
# nodeリスト
nodes = sorted(set([word for pair in jaccard_dict.keys() for word in pair]))

c_word ={}

for r, n in enumerate(nodes):
  c_word.update({n : dict_c[n]})

G = nx.Graph()

#  接点／単語（node）の追加
G.add_nodes_from(nodes)
print('Number of nodes: {}'.format(G.number_of_nodes()))

#  線（edge）の追加
for pair, coef in jaccard_dict.items():
    G.add_edge(pair[0], pair[1], weight=coef)
print('Number of edges: {}'.format(G.number_of_edges()))

# nodeの配置方法の指定
seed = 0
np.random.seed(seed)
pos = nx.nx_agraph.graphviz_layout(
    G,
    prog='neato',
    args='-Goverlap="scalexy" -Gsep="+6" -Gnodesep=0.8 -Gsplines="polyline" -GpackMode="graph" -Gstart={}'.format(seed))

# nodeの色をページランクアルゴリズムによる重要度により変える
pr = nx.pagerank(G)

# インタラクティブな共起ネットワークの可視化
build_interactive_network(G, pos, list(c_word.values()), list(pr.values()))

Number of nodes: 197
Number of edges: 642
